# Example stitching workflow

Using a small CZI example dataset with two positions.

This notebook requires multiview-stitcher to be installed as such:
- `pip install ".[aicsimageio]"`
- `pip install "multiview-stitcher[aicsimageio]"`

In [1]:
# napari-stitcher can be used to visualize the stitching process using napari
VISUALIZE_USING_NAPARI = False

In [2]:
import dask.diagnostics

from multiview_stitcher import (
    io,
    sample_data,
    msi_utils,
    registration,
    fusion,
)

if VISUALIZE_USING_NAPARI:
    from napari_stitcher import viewer_utils
    import napari
    viewer = napari.Viewer()

In [ ]:
# Load dataset from file

filename = sample_data.get_mosaic_sample_data_path()

sims = io.read_mosaic_into_sims(filename)
msims = [msi_utils.get_msim_from_sim(sim, scale_factors=[]) for sim in sims]

print(f"Loaded {len(msims)} positions from {filename}. Transform key is {io.METADATA_TRANSFORM_KEY}.")

Loaded 2 positions from /Users/malbert/software/multiview-stitcher/src/multiview_stitcher/test-datasets/mosaic_test.czi. Transform key is affine_metadata.


In [4]:
if VISUALIZE_USING_NAPARI:
    
    layer_tuples = viewer_utils.create_image_layer_tuples_from_msims(
        msims, n_colors=3, transform_key='affine_metadata')
    viewer_utils.add_image_layer_tuples_to_viewer(viewer, layer_tuples)
    viewer.reset_view()

In [5]:
print('Registering...')
with dask.diagnostics.ProgressBar():
    params = registration.register(
        msims, transform_key='affine_metadata', reg_channel_index=0)
    
for msim, param in zip(msims, params):
    msi_utils.set_affine_transform(msim, param, transform_key='affine_registered', base_transform_key='affine_metadata')

print('Fusing...')
with dask.diagnostics.ProgressBar():
    fused = fusion.fuse(
        [msi_utils.get_sim_from_msim(msim) for msim in msims],
        transform_key='affine_registered')

mfused = msi_utils.get_msim_from_sim(fused, scale_factors=[])

Registering...
[########################################] | 100% Completed | 948.16 ms
[########################################] | 100% Completed | 212.24 ms
Fusing...


In [6]:
if VISUALIZE_USING_NAPARI:
    
    viewer = napari.Viewer(ndisplay=2)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims, transform_key='affine_registered', n_colors=4,
        name_prefix='pre-registered view')
    mlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

    lds = viewer_utils.create_image_layer_tuples_from_msim(
        mfused, transform_key='affine_registered',
        name_prefix='registered view')
    rlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

In [7]:
with dask.diagnostics.ProgressBar():
    io.save_sim_as_tif('fused.tif', fused)

  0%|          | 0/1.0 [00:00<?, ?it/s]

[                                        ] | 0% Completed | 78.83 us

[########################################] | 100% Completed | 386.25 ms


100%|██████████| 1/1.0 [00:00<00:00,  2.56it/s]
